In [1]:
!pip install -U --pre pycaret

In [4]:
import numpy as np
import pandas as pd

from pycaret.regression import *
from pycaret.utils import *
from sklearn.metrics import mean_squared_error

In [5]:
dataset = pd.read_csv('datasets/big-table-train.csv')
dataset = dataset.dropna()
dataset = dataset.drop(columns=['Id', 'completed_time'])
dataset

,running_time,route_distance_km,delta_time,node_start,node_finish,distance,speed,average_speed,lon_start,lat_start,lon_end,lat_end
0,2022-01-24 18:30:21,3.740,862.0,10980417,4768348185,31.771489,30.0,29.537736,30.738275,46.468524,30.738097,46.468266
3,2022-01-24 18:30:21,3.740,862.0,10980431,5125489557,50.034390,29.0,29.537736,30.739016,46.471232,30.738957,46.470784
4,2022-01-24 18:30:21,3.740,862.0,10980432,5213300015,13.453126,30.0,29.537736,30.739248,46.472665,30.739226,46.472545
5,2022-01-24 18:30:21,3.740,862.0,10980471,2133368054,15.994386,32.0,29.537736,30.739472,46.474116,30.739449,46.473965
7,2022-01-24 18:30:21,3.740,862.0,10980545,5213015312,47.897295,30.0,29.537736,30.740532,46.481082,30.740463,46.480654
...,...,...,...,...,...,...,...,...,...,...,...,...
401294,2022-01-24 14:57:46,2.323,819.0,5225320781,6570497841,11.114741,18.0,23.222222,30.728533,46.410472,30.728391,46.410452
401295,2022-01-24 14:57:46,2.323,819.0,6570497841,352681459,13.232601,21.0,23.222222,30.728391,46.410452,30.728222,46.410428
401296,2022-01-24 14:57:46,2.323,819.0,6570497861,1986778509,41.040972,17.0,23.222222,30.723430,46.409822,30.723333,46.410186
401297,2022-01-24 14:57:46,2.323,819.0,6573889998,312712557,20.488592,19.0,23.222222,30.726552,46.417987,30.726339,46.417987


In [6]:
data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (297058, 12)
Unseen Data For Predictions: (33007, 12)


In [7]:
s = setup(data = data, target = 'delta_time', session_id=123)

,Description,Value
0,Session id,123
1,Target,delta_time
2,Target type,Regression
3,Original data shape,"(297058, 12)"
4,Transformed data shape,"(297058, 12)"
5,Transformed train set shape,"(207940, 12)"
6,Transformed test set shape,"(89118, 12)"
7,Numeric features,10
8,Categorical features,1
9,Preprocess,True


In [8]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [9]:
best_model = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,6.6395,372.3861,19.2973,0.9908,0.0358,0.0117
1,6.7742,387.8183,19.6931,0.9905,0.0388,0.0123
2,6.5571,352.7787,18.7824,0.9912,0.0352,0.0115
3,6.5912,376.7328,19.4096,0.9906,0.0377,0.0118
4,6.4346,349.3914,18.6920,0.9913,0.0346,0.0113
5,6.8558,409.1719,20.2280,0.9898,0.0386,0.0122
6,6.4767,344.6302,18.5642,0.9914,0.0355,0.0115
7,6.6858,383.1623,19.5745,0.9906,0.0428,0.0127
8,6.6299,373.6174,19.3292,0.9908,0.0363,0.0117


In [10]:
tuned_best_model = tune_model(best_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.9600,373.7111,19.3316,0.9907,0.0388,0.0178
1,10.1208,406.4644,20.1610,0.9900,0.0409,0.0183
2,9.9321,332.6267,18.2381,0.9917,0.0363,0.0176
3,9.9663,393.8163,19.8448,0.9902,0.0398,0.0177
4,9.4530,313.4028,17.7032,0.9922,0.0349,0.0168
5,9.9506,376.2973,19.3984,0.9906,0.0396,0.0178
6,9.8035,382.9125,19.5682,0.9904,0.0405,0.0177
7,9.9025,391.1123,19.7766,0.9904,0.0450,0.0186
8,9.8920,365.2733,19.1121,0.9910,0.0392,0.0178


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [11]:
# validate on unseen data
unseen_predictions = predict_model(tuned_best_model, data=data_unseen)
unseen_predictions

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,9.0591,211.8790,14.5561,0.9947,0.0295,0.0159


,running_time,route_distance_km,node_start,node_finish,distance,speed,average_speed,lon_start,lat_start,lon_end,lat_end,delta_time,prediction_label
0,2022-01-24 18:30:21,3.740,290891853,4441087524,59.342655,22.0,29.537735,30.736652,46.482773,30.737408,46.482719,862.0,856.943827
1,2022-01-24 18:30:21,3.740,2479719949,6806370974,18.176003,16.0,29.537735,30.732115,46.456554,30.732332,46.456490,862.0,845.489247
2,2022-01-24 18:30:21,3.740,4786018208,4845924947,30.463514,42.0,29.537735,30.733671,46.463360,30.733461,46.463127,862.0,844.728967
3,2022-01-24 18:30:21,3.740,5213300078,289702925,35.789761,31.0,29.537735,30.740143,46.478451,30.740093,46.478130,862.0,863.990974
4,2022-01-24 18:30:21,3.740,5214555660,6583283185,37.305794,34.0,29.537735,30.737041,46.467083,30.736790,46.466797,862.0,850.302635
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33002,2022-01-24 14:57:46,2.323,352682638,4374904173,155.209946,20.0,23.222221,30.728752,46.418114,30.726734,46.417999,819.0,800.529056
33003,2022-01-24 14:57:46,2.323,2142884470,352682638,95.333694,25.0,23.222221,30.729990,46.418190,30.728752,46.418114,819.0,795.914618
33004,2022-01-24 14:57:46,2.323,3912884426,352681477,37.623245,23.0,23.222221,30.727289,46.410301,30.726809,46.410240,819.0,794.843221
33005,2022-01-24 14:57:46,2.323,4775806433,352666805,95.083893,27.0,23.222221,30.737497,46.418636,30.735846,46.418541,819.0,794.751656


In [12]:
mean_squared_error(unseen_predictions['delta_time'], unseen_predictions['prediction_label'], squared=False)

14.556064133911347

### Model predictions on test data

In [13]:
big_table_test_df = pd.read_csv("datasets/big-table-test.csv")

In [14]:
submission_df = pd.read_csv("datasets/submission.csv")
submission_df

,Id,Predicted
0,4567450496676323264,698.387007
1,-1762687449608244695,700.759720
2,558209096572647780,652.301603
3,1091960336272216078,659.229188
4,-1321994599972009663,595.579733
...,...,...
995,-1019532991501300185,591.330224
996,-3501571252544462083,555.108671
997,8305010046033518305,692.395130
998,5747749188503396733,741.265722


In [15]:
submission_predictions = predict_model(tuned_best_model, data=big_table_test_df)
submission_predictions

,Id,node_start,node_finish,distance,speed,running_time,route_distance_km,average_speed,lon_start,lat_start,lon_end,lat_end,prediction_label
0,-2627062893189810184,10980432,2133368107,17.414917,32.0,2022-01-24 17:21:01,6.161,31.155064,30.739248,46.472664,30.739021,46.472683,688.033811
1,-2627062893189810184,10980433,5212387954,17.186539,26.0,2022-01-24 17:21:01,6.161,31.155064,30.741320,46.472507,30.741096,46.472523,687.401400
2,-2627062893189810184,10980445,5221700954,28.513481,26.0,2022-01-24 17:21:01,6.161,31.155064,30.747110,46.472084,30.746740,46.472115,695.364746
3,-2627062893189810184,10980498,10980445,154.266129,25.0,2022-01-24 17:21:01,6.161,31.155064,30.749115,46.471951,30.747110,46.472084,695.441171
4,-2627062893189810184,10980647,1986137911,8.542824,29.0,2022-01-24 17:21:01,6.161,31.155064,30.753937,46.473095,30.753922,46.473019,698.830967
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78975,3680089850189149544,8952405843,8952405842,33.190815,29.0,2022-01-24 19:17:17,3.615,29.081522,NaN,NaN,NaN,NaN,680.383559
78976,3680089850189149544,8952405844,8952405843,81.341690,25.0,2022-01-24 19:17:17,3.615,29.081522,NaN,NaN,NaN,NaN,680.853904
78977,3680089850189149544,8952405845,8952405844,65.715019,38.0,2022-01-24 19:17:17,3.615,29.081522,NaN,NaN,NaN,NaN,677.165205
78978,3680089850189149544,8952405846,8952405845,247.167236,37.0,2022-01-24 19:17:17,3.615,29.081522,NaN,NaN,NaN,NaN,675.814651


In [16]:
predicted_df = submission_predictions[['Id', "prediction_label"]].drop_duplicates(subset=["Id"], keep='first')
predicted_df

,Id,prediction_label
0,-2627062893189810184,688.033811
158,3426541173123240405,859.625765
270,-3474192753915350461,705.087572
408,3538643101914165609,634.991598
505,-8718775480721360374,653.530136
...,...,...
78582,4250591723952515712,689.903735
78697,6641928463975996922,574.001312
78756,-3513599972731888784,430.657055
78803,3062492146546450059,696.470274


In [17]:
submission_df = submission_df \
    .merge(predicted_df, on='Id', how='inner') \
    .drop_duplicates(subset=["Id"], keep='first')
submission_df

,Id,Predicted,prediction_label
0,4567450496676323264,698.387007,698.387007
1,-1762687449608244695,700.759720,700.759720
2,558209096572647780,652.301603,652.301603
3,1091960336272216078,659.229188,659.229188
4,-1321994599972009663,595.579733,595.579733
...,...,...,...
995,-1019532991501300185,591.330224,591.330224
996,-3501571252544462083,555.108671,555.108671
997,8305010046033518305,692.395130,692.395130
998,5747749188503396733,741.265722,741.265722


In [18]:
submission_df["Predicted"] = submission_df["prediction_label"]
submission_df.drop("prediction_label", axis=1, inplace=True)

In [19]:
submission_df.to_csv("datasets/submission.csv", index=False)

## Submission phase

Guide: https://www.kaggle.com/code/derrickmwiti/how-to-make-submissions-using-kaggle-s-api/notebook

In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ["KAGGLE_USERNAME"]="beardimon"
os.environ["KAGGLE_KEY"]="22adc3776f244a54f4732bf8dfb077eb"

In [ ]:
!kaggle competitions submit -c int20h-2023-hackathon  -f ./datasets/submission.csv -m "Baseline"

In [ ]:
!kaggle competitions leaderboard -c int20h-2023-hackathon --show

In [ ]:
!kaggle competitions submissions -c int20h-2023-hackathon